## Modeling 
The goal of this notebook is to confirm what model will be used for the offensive coordinator and ensure it can be effectively implemented with the data from 01_exploration before preprocessing steps are ported to C++. Eventually the modeling steps in this notebook will be done in a train.py file.   

**Tasks:**  
1. Load Cleaned Data
2. Model Data
3. Evaluate Model

**Note For Part 2**  
The two types of models that will be compared/evalutated for use in the final training script are XGBoost and Random Forest. For this kind of classification problem which includes nonlinear interactions and many contextual dependencies it is likely that XGBoost will perform better due to its ability to handle nonlinear/contextual relationships better and catch rare but important situations. (Example: hail mary attempts when losing in the last minute of a football game)

**1. Load Cleaned Data**

In [9]:
import pandas as pd

In [11]:
df = pd.read_csv("C:/Users/Tyler/OneDrive/Coding/Repositories/AI_Offensive_Coordinator-/python/data/cleaned_pbp_22_to_24.csv")

In [13]:
df.head()

,season,week,game_id,posteam,defteam,qtr,down,ydstogo,yardline_100,score_differential,def_pass_rate_last3,avg_yds_pp_alwd_last3,avg_rush_pp_alwd_last3,play_subtype
0,2022,1,2022_01_BAL_NYJ,NYJ,BAL,1.0,1.0,10.0,78.0,0.0,0.656250,5.358173,4.320055,outside run
1,2022,1,2022_01_BAL_NYJ,NYJ,BAL,1.0,1.0,10.0,59.0,0.0,0.656250,5.358173,4.320055,short pass left
2,2022,1,2022_01_BAL_NYJ,NYJ,BAL,1.0,2.0,10.0,59.0,0.0,0.656250,5.358173,4.320055,outside run
3,2022,1,2022_01_BAL_NYJ,NYJ,BAL,1.0,3.0,5.0,54.0,0.0,0.656250,5.358173,4.320055,short pass right
4,2022,1,2022_01_BAL_NYJ,BAL,NYJ,1.0,1.0,10.0,72.0,0.0,0.488917,5.883399,3.732620,short pass right


In [15]:
df.isnull().sum()

season                    0
week                      0
game_id                   0
posteam                   0
defteam                   0
qtr                       0
down                      0
ydstogo                   0
yardline_100              0
score_differential        0
def_pass_rate_last3       0
avg_yds_pp_alwd_last3     0
avg_rush_pp_alwd_last3    0
play_subtype              0
dtype: int64